In [ ]:
!pip3 install pynrrd
!pip3 install pyradiomics

import pydicom as dicom
import numpy as np
import matplotlib.pylab as plt
import nibabel as nib
import scipy
import dicom2nifti

import time
import os
import csv

from skimage.transform import resize
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

import cv2
import numpy as np

# Import Densenet from Keras
from keras.applications.densenet import DenseNet121
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Model
from keras import backend as K

from scipy.ndimage import zoom
from numpy import save, load

import tensorflow as tf
import math
import SimpleITK as sitk
import itk
import itkwidgets
import nrrd

import six
import radiomics
from radiomics import featureextractor, getFeatureClasses

PATCH_SIZE_X=32
PATCH_SIZE_Y=32
PATCH_SIZE_Z=32
COMPRESSION_RATIO=.125
DICOM_UNIFORM_ROWS=64
DICOM_UNIFORM_COLUMNS=64
DICOM_UNIFORM_SLICES=32

mgmt_dictionary = {}
with open('/Users/user/Desktop/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    fields = next(csvreader)
    for row in csvreader:        
        mgmt_dictionary[row[0]] = int(row[1])
print(mgmt_dictionary)

paramsFile = os.path.abspath(r'testdata.yaml')
extractor = featureextractor.RadiomicsFeatureExtractor(paramsFile)

# Initialize feature extractor using the settings file
def create_patches(image_data_modality_t1c, image_data_modality_flair):
    print("Creating patches")
    patches = []
    patch_locations = []
    width, height, depth = image_data_modality_t1c.shape
    nx_patches = width/PATCH_SIZE_X
    ny_patches = height/PATCH_SIZE_Y
    nz_patches = depth/PATCH_SIZE_Z
    center_x = width/2
    center_y = height/2
    center_z = depth/2
    for x in range(0,width,PATCH_SIZE_X):
        for y in range(0,height,PATCH_SIZE_Y):
            for z in range(0,depth,PATCH_SIZE_Z):
                if (((z+PATCH_SIZE_Z) > depth) or ((y+PATCH_SIZE_Y) > height) or ((x+PATCH_SIZE_X) > width)):
                    break
                    
                patch = np.empty((4,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            patch[0][px][py][pz] = image_data_modality_flair[x+px][y+py][z+pz]
                            
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            minimum = 99999
                            for mx in [-1,0,1]:
                                for my in [-1,0,1]:
                                    for mz in [-1,0,1]:
                                        if ((px+mx >=0) and (px+mx < PATCH_SIZE_X)
                                            and (py+my >=0) and (py+my < PATCH_SIZE_Y)
                                            and (pz+mz >=0) and (pz+mz < PATCH_SIZE_Z)):
                                            if minimum > patch[0][px+mx][py+my][pz+mz]:
                                                minimum = patch[0][px+mx][py+my][pz+mz]
                            patch[1][px][py][pz] = minimum
                            
                flair = np.empty((PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            flair[px][py][pz] = image_data_modality_flair[x+px][y+py][z+pz]

                # Use pyradiomics to get NGTM
                imageName = 'testdata.nrrd'
                nrrd.write(imageName, np.array(flair))
                maskName = "testdata_mask.nrrd"
                mask_data = np.ones((PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                mask_data[PATCH_SIZE_X-2][PATCH_SIZE_Y-2][PATCH_SIZE_Z-2] = 0
                nrrd.write(maskName, mask_data)
                
                # print("Starting to extracting features")
                featureVector = extractor.execute(imageName, maskName, voxelBased=True)
                # print("Completed extracting features")
                
                ngtdm_data = np.empty((width, height, depth))
                for featureName, featureValue in six.iteritems(featureVector):
                    if featureName == "original_ngtdm_Coarseness":
                        # print('%s: %s' % (featureName, featureValue))
                        if isinstance(featureValue, sitk.Image):
                            ngtdm_file = featureName + '.nrrd'
                            sitk.WriteImage(featureValue, ngtdm_file)
                            # print('Computed %s, stored as %s' % (featureName, ngtdm_file))
                            ngtdm_data, header = nrrd.read(ngtdm_file)

                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            patch[2][px][py][pz] = ngtdm_data[px][py][pz]
                            distance = math.sqrt((x+px-center_x)**2 + (y+py-center_y)**2 + (z+pz-center_z)**2) 
                            patch[3][px][py][pz] = distance
                            
                patches.append(patch)
                patch_locations.append(z<<18|y<<9|x)
                # print("patch location: %d" % (z<<18|y<<9|x))
                # print("patch count: %d" % len(patches))
            if y+PATCH_SIZE_Y > height:
                break
        if x+PATCH_SIZE_X > width:
            break
    print("Final patch count: %d" % len(patches))
    return patches, patch_locations

def save_patches(patient_id, patches, patch_locations):
    print("Saving patches for patient %s", patient_id)
    for patch_num, patch in enumerate(patches):
        patch_array = np.array(patch)
        output_file = 'DICOM/dicom-' + patient_id + '-' + str(patch_locations[patch_num]) + '.npy'
        save(output_file, patch_array)
    return patches


def find_first_z(image_data):
    width, height, depth = image_data.shape
    min_z = 0
    for z in range(0,depth):
        for x in range(0,width):
            for y in range(0,height):
                if image_data[x][y][z] != 0:
                    min_z = z;
                    return min_z
    return min_z

def find_last_z(image_data):
    width, height, depth = image_data.shape
    max_z = 0
    for z in range(depth-1,0,-1):
        for x in range(0,width):
            for y in range(0,height):
                if image_data[x][y][z] != 0:
                    max_z = z;
                    return max_z
    return max_z

def find_first_x(image_data):
    width, height, depth = image_data.shape
    min_x = 0
    for x in range(0,width):
        for y in range(0,height):
            for z in range(0,depth):
                if image_data[x][y][z] != 0:
                    min_x = x;
                    return min_x
    return min_x
            
def find_last_x(image_data):
    width, height, depth = image_data.shape
    max_x = 0
    for x in range(width-1,0,-1):
        for y in range(0,height):
            for z in range(0,depth):
                if image_data[x][y][z] != 0:
                    max_x = x;
                    return max_x

def find_first_y(image_data):
    width, height, depth = image_data.shape
    min_y = 0
    for y in range(0,height):
        for x in range(0,width):
            for z in range(0,depth):
                if image_data[x][y][z] != 0:
                    min_y = y;
                    return min_y
            
def find_last_y(image_data):
    width, height, depth = image_data.shape
    max_y = 0
    for y in range(height-1,0,-1):
        for x in range(0,width):
            for z in range(0,depth):
                if image_data[x][y][z] != 0:
                    max_y = y;
                    return max_y

def remove_black_slices(image_data):
    min_x = find_first_x(image_data)
    max_x = find_last_x(image_data)
    min_y = find_first_y(image_data)
    max_y = find_last_y(image_data)
    min_z = find_first_z(image_data)
    max_z = find_last_z(image_data)
    print("(%d,%d,%d)-(%d,%d,%d)" % (min_x,min_y,min_z,max_x,max_y,max_z))
    pruned_image = np.empty((max_x-min_x+1,max_y-min_y+1,max_z-min_z+1))
    for x in range(min_x,max_x+1):
        for y in range(min_y,max_y+1):
            for z in range(min_z,max_z+1):
                pruned_image[x-min_x][y-min_y][z-min_z]=image_data[x][y][z]
    return pruned_image

def remove_black_borders(image):
    x_nonzero, y_nonzero, z_nonzero = np.nonzero(image)
    cropped_image = image[np.min(x_nonzero):np.max(x_nonzero), np.min(y_nonzero):np.max(y_nonzero), np.min(z_nonzero):np.max(z_nonzero)]
    print("Image dimension ")
    print(image.shape)
    print("Cropped image dimension ")
    print(cropped_image.shape)
    return cropped_image

def load_patch(patient_id, patch_num):
    print("Loading patch, Patient id: %s, Patch num: %d" % (patient_id, patch_num))
    patch_file = 'Dicom/dicom_' + patient_id + '_' + str(patch_num) + '.npy'
    patch_array = np.load(patch_file)
    print(patch_array)
 
min_depth = 99999
min_width = 99999
min_height = 99999
found_732 = False
for patient_folder in os.listdir('/Users/user/Desktop/rsna-miccai-brain-tumor-radiogenomic-classification/train/'):
    if (patient_folder == '.DS_Store'):
        continue
    image_path = '/Users/user/Desktop/rsna-miccai-brain-tumor-radiogenomic-classification/train/'+patient_folder+'/T1wCE/'
    dicom2nifti.convert_directory(image_path, image_path)
    for name in os.listdir(image_path):
        if 'nii' in name:
            nifti_file = image_path + name
            break
    image_obj=nib.load(nifti_file)
    image_data_modality_t1c=image_obj.get_fdata()
    print("Before removing black borders: TLC shape: ")
    print(image_data_modality_t1c.shape)
    image_data_modality_t1c_1=remove_black_slices(image_data_modality_t1c)
    print("After removing black slices: TLC shape: ")
    print(image_data_modality_t1c_1.shape)
    image_data_modality_t1c_2=remove_black_slices(image_data_modality_t1c)
    print("After removing black borders: TLC shape: ")
    print(image_data_modality_t1c_2.shape)
    image_path = '/Users/user/Desktop/rsna-miccai-brain-tumor-radiogenomic-classification/train/'+patient_folder+'/FLAIR/'
    dicom2nifti.convert_directory(image_path, image_path)
    for name in os.listdir(image_path):
        if 'nii' in name:
            nifti_file = image_path + name
            break
    image_obj=nib.load(nifti_file)
    image_data_modality_flair=image_obj.get_fdata()
    print("Before removing black borders: FLAIR shape: ")
    print(image_data_modality_flair.shape)
    image_data_modality_flair_1=remove_black_slices(image_data_modality_flair)
    print("After removing black slices: FLAIR shape: ")
    print(image_data_modality_flair_1.shape)
    image_data_modality_flair_2=remove_black_borders(image_data_modality_flair)
    print("After removing black borders: FLAIR shape: ")
    print(image_data_modality_flair_2.shape)    
    width, height, depth = image_data_modality_flair_2.shape
    if depth < min_depth:
        min_depth = depth
    if height < min_height:
        min_height = height
    if width < min_width:
        min_width = width
    if (((width%PATCH_SIZE_X) > int(.10*width)) or ((height%PATCH_SIZE_Y) > int(.10*height)) or ((depth%PATCH_SIZE_Z) > int(.10*depth))):
        continue
    ''' TBU:    
    patches, patch_locations = create_patches(image_data_modality_t1c, image_data_modality_flair)
    print("number of patches %d" % (len(patches)))
    save_patches(patient_folder, patches, patch_locations)
    TBU: '''
    print("Patient-id %s" % (patient_folder))
print("min depth %d, min width %d, min height %d" % (min_depth, min_width, min_height))